In [95]:
from numexpr.expressions import double
from sympy import *
import numpy as np
import math as math
import sympy as sp

In [96]:
la,wa,ta,E,G,J,lb,wb,tb,lc,wc,tc,l,w,t,psi,d = symbols('la,wa,ta,E,G,J,lb,wb,tb,lc,wc,tc,l,w,t,psi,d')

In [97]:
Ca = Matrix([[0, 0, 0, la/(G*J), 0, 0],
             [0, 0, -6*la**2/(ta*wa**3*E), 0, 12*la/(ta*wa**3*E), 0],
             [0, 6*la**2/(ta**3*wa*E), 0, 0, 0, 12*la/(ta**3*wa*E)],
             [la/(ta*wa*E), 0, 0, 0, 0, 0],
             [0, 4*la**3/(ta**3*wa*E), 0, 0, 0, 6*la**2/(ta**3*wa*E)],
             [0, 0, 4*la**3/(ta*wa**3*E), 0, -6*la**2/(ta*wa**3*E), 0]])

Cb = Matrix([[0, 0, 0, lb/(G*J), 0, 0],
             [0, 0, -6*lb**2/(wb*tb**3*E), 0, 12*lb/(wb*tb**3*E), 0],
             [0, 6*lb**2/(wb**3*tb*E), 0, 0, 0, 12*lb/(wb**3*tb*E)],
             [lb/(wb*tb*E), 0, 0, 0, 0, 0],
             [0, 4*lb**3/(wb**3*tb*E), 0, 0, 0, 6*lb**2/(wb**3*tb*E)],
             [0, 0, 4*lb**3/(wb*tb**3*E), 0, -6*lb**2/(wb*tb**3*E), 0]])

Cc = Matrix([[0, 0, 0, lc/(G*J), 0, 0],
             [0, 0, -6*lc**2/(tc*wc**3*E), 0, 12*lc/(tc*wc**3*E), 0],
             [0, 6*lc**2/(tc**3*wc*E), 0, 0, 0, 12*lc/(tc**3*wc*E)],
             [lc/(tc*wc*E), 0, 0, 0, 0, 0],
             [0, 4*lc**3/(tc**3*wc*E), 0, 0, 0, 6*lc**2/(tc**3*wc*E)],
             [0, 0, 4*lc**3/(tc*wc**3*E), 0, -6*lc**2/(tc*wc**3*E), 0]])

R = eye(3)

Da = Matrix([[0,0,0],
             [0,0, lb+lc],
             [0,-lb-lc,0]])

Db = Matrix([[0,0,0],
             [0,0,lc],
             [0,-lc,0]])

Ad_a = Matrix([[R, zeros(3)],
               [Da*R, R]])

Ad_b = Matrix([[R, zeros(3)],
               [Db*R, R]])

Ct = Ad_a*Ca*Ad_a.inv()+Ad_b*Cb*Ad_b.inv()+Cc
Ct_Assumed = simplify(Ct.subs({ta:t,tb:t,tc:t,wa:w,wb:w,wc:w,la:l,lb:l,lc:l}))

R1 = Matrix([[cos((pi+psi)/2), -sin((pi+psi)/2), 0],
             [sin((pi+psi)/2), cos((pi+psi)/2), 0],
             [0, 0, 1]])
#display(R1)

R2 = Matrix([[cos((pi-psi)/2), -sin((pi-psi)/2), 0],
             [sin((pi-psi)/2), cos((pi-psi)/2), 0],
             [0, 0, 1]])
#display(R2)

D1 = Matrix([[0, 0, 0],
             [0, 0, -d/2],
             [0, d/2, 0]])
#display(D1)

D2 = Matrix([[0, 0, 0],
             [0, 0, d/2],
             [0, -d/2, 0]])
#display(D2)


Ad_1 = Matrix([[R1, zeros(3)],
               [D1*R1, R1]])
#display(Ad_1)

Ad_2 = Matrix([[R2, zeros(3)],
               [D2*R2, R2]])
#display(Ad_2)

Kt_Assumed = cancel(Ct_Assumed.inv())
Kp_Assumed = cancel(Ad_1*Kt_Assumed*Ad_1.inv() + Ad_2*Kt_Assumed*Ad_2.inv())

#Abbreviating Elements for Display
m, n = Kp_Assumed.shape
C = Matrix(m, n, lambda i, j: Symbol(f'k{i+1}{j+1}'))

# Loop through each element and replace non-zero entries with placeholders
for i in range(m):
    for j in range(n):
        if Kp_Assumed[i, j] == 0:
            C[i, j] = 0  # Retain zeros

In [98]:
Load = Matrix([300,-588.6,300,0,80,0])
t0,w0,l0,d0,psi0,E0,G0 = 1e-3,10e-3,20e-3,20e-3,math.radians(60),125e9,48e9
J0 = 0.312*w0*t0**3
Volume = t0*w0*l0*3*2 + t0*w0**2*4 + w0*d0**2
Mass = 4480*Volume
y_height = (3*l0+t0*4)*cos(psi0/2)+d0/2
x_width = (3*l0+t0*3)*sin(psi0/2)*2+d0
z_width = w0

Kp_Assumed_num = Kp_Assumed.subs({t:t0, w:w0, l:l0, d:d0, psi:psi0, E:E0, G:G0, J:J0})
Cp_Assumed_num = Kp_Assumed_num.inv()
freq_x = float(1/(2*pi*sqrt(Mass*Cp_Assumed_num[4-1,1-1])))
freq_z = float(1/(2*pi*sqrt(Mass*Cp_Assumed_num[6-1,3-1])))

Displacement = Kp_Assumed_num.solve(Load)

print(f"Mass = {Mass*1000:.2f} g")
print(f"X Size = {x_width*1000:.2f} mm\nY Size = {y_height*1000:.2f} mm\nZ Size = {z_width*1000:.2f} mm")
print(f"Natural Frequency in X Axis = {freq_x:.2f} Hz\nNatural Frequency in Z Axis = {freq_z:.2f} Hz")
print(f"X Displacement = {Displacement[3]*1000:.4f} mm\nY Displacement = {Displacement[4]*1000:.4f} mm\nZ Displacement = {Displacement[5]*1000:.4f} mm\nX Rotation = {math.degrees(Displacement[0]):.4f} deg\nY Rotation = {math.degrees(Displacement[1]):.4f} deg\nZ Rotation = {math.degrees(Displacement[2]):.4f} deg")

Mass = 25.09 g
X Size = 83.00 mm
Y Size = 65.43 mm
Z Size = 10.00 mm
Natural Frequency in X Axis = 343.21 Hz
Natural Frequency in Z Axis = 112.47 Hz
X Displacement = 2.5714 mm
Y Displacement = -0.0188 mm
Z Displacement = 23.9437 mm
X Rotation = 50.1359 deg
Y Rotation = 28.4158 deg
Z Rotation = 8.4352 deg
